<a href="https://colab.research.google.com/github/ashwinsinha32/Project1Redo/blob/main/Project1_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Name: Ashwin Sinha

# Project \#1 Starter Code: 
Your first step is to decide which dataset you would like to work with for this project. You can find the data on Blackboard. 

I will be using the Ramen dataset:

- [Ramen Ratings](https://www.kaggle.com/residentmario/ramen-ratings)


In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
happy =pd.read_csv('/content/drive/MyDrive/CS167Spring22/datasets/happiness_data.csv')
happy.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Overall rank,Year,Country or region,Score,GDP per capita,Social support and Family,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,2018,Finland,7.632,1.305,1.592,0.874,0.681,0.202,0.393
1,2,2018,Norway,7.594,1.456,1.582,0.861,0.686,0.286,0.340
2,3,2018,Denmark,7.555,1.351,1.590,0.868,0.683,0.284,0.408
3,4,2018,Iceland,7.495,1.343,1.644,0.914,0.677,0.353,0.138
4,5,2018,Switzerland,7.487,1.420,1.549,0.927,0.660,0.256,0.357


# Problem: 
Which dataset are you going to use? What questions are you going to ask about this dataset? Make sure that you can answer the question using ML models. What are your predictor variables and target variable?

I will be using the Happiness dataset to determine which countries have the highest happiness score given the numerous features we have available, from generosity levels to life expectancy predictions. With this information, we can evaluate a country's happiness ranking, along with predicting the general pleasure one might gain from living in a country. Target Variable = Score |
Predictor Variables = GDP per capita, generosity, freedom to make life choices, and social support and family.

# Data Preparation
Explain your data prep. What did you do to get your data in the shape for your experiments? (e.g. creation of dummy variables, filling in missing values, etc). 

The original dataset has 2,580 rows, but after dropping rops that contained a value of "Unrated" in the Stars category, we reduced it down to 2,577 rows.  We then needed to convert NaN values in the "Top Ten" category to zero to do our analysis.  Additionally, I created a couple different subsets based on different predictors as well as a subset that contained all the values of the highest rating "Top Ten" values since 2012.  This subset called TopRamen contains 37 rows.  TopRamenHQ shows which countries the best ramen are from, along with the brand and a couple other features.  Lastly, we created dummy variables to give binary nunmbers to the brand and style of ramen so that we can convert them to a quantitative figure.

In [3]:
# data prep code 
happy.shape

(782, 10)

In [6]:
happy['GDP per capita'].isnull().values.any()
happy['Social support and Family'].isnull().values.any()
happy['Generosity'].isnull().values.any()
happy['Freedom to make life choices'].isnull().values.any()


print("after: ", happy.shape)

after:  (782, 10)


In [12]:
Happy2 = happy.dropna( subset = ['Country or region']).copy()
print("after: ", Happy2.shape)

after:  (782, 10)


In [11]:
CleanHappy = happy[CleanHappy["Year"].astype(str).str.contains('2018')]
CleanHappy.head()

,Overall rank,Year,Country or region,Score,GDP per capita,Social support and Family,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,2018,Finland,7.632,1.305,1.592,0.874,0.681,0.202,0.393
1,2,2018,Norway,7.594,1.456,1.582,0.861,0.686,0.286,0.340
2,3,2018,Denmark,7.555,1.351,1.590,0.868,0.683,0.284,0.408
3,4,2018,Iceland,7.495,1.343,1.644,0.914,0.677,0.353,0.138
4,5,2018,Switzerland,7.487,1.420,1.549,0.927,0.660,0.256,0.357


# Metrics
Identify which metrics you will be using to test your model and say why they are appropriate.

We will be doing a regression algorithm to determine how the machine can assess happiness levels agains the overall ranking in column 1.  We will use the r^2 value to assess the performance of the models.  This is appropriate as r^2 values can measure regression models instead of using accuracy.

# Baseline Performance
Identify a baseline for your metrics - what would you expect to happen with this particular data if you always guessed the average or if you guessed randomly? What did some of the unsuccessful machine learning algorithms yield?

The baseline for our metrics is around an accuracy score of .0174.  This is a dummy classifier algorithm that removes Country from the dataset, and then evaluates its accuracy on how well the model did to predict which country a new ramen review is from.  This is the "unsmart" model that would happen if you guessed randomly each time.

In [18]:
# build and test a baseline model here
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics

predictors = ['GDP per capita', 'Social support and Family', 'Freedom to make life choices', 'Generosity']
target = 'Score'

train_data, test_data, train_sln, test_sln = train_test_split(happy[predictors], happy[target], test_size = 0.2, random_state = 41)
train_data.head()

baseline = DummyRegressor()
baseline.fit(train_data, train_sln)
baseline_pred = baseline.predict(test_data)

print('r^2: ', metrics.r2_score(test_sln, baseline_pred))

r^2:  -0.0005261696220115741


# Model planning and Execution
Identify which learning algorithms you have tried and which important parameters you’ve tuned for each one.


In [21]:
# Execute your plan here (build and test your models to answer your question)

# knn
import numpy as np
from sklearn import neighbors

knn = neighbors.KNeighborsRegressor(n_neighbors=5)
knn.fit(train_data,train_sln)
happy_preds = knn.predict(test_data)
print('r^2: ', metrics.r2_score(test_sln, happy_preds))

r^2:  0.7338162515822132


In [23]:
#Weighted knn
wknn = neighbors.KNeighborsRegressor(n_neighbors=5, weights = 'distance')
wknn.fit(train_data,train_sln)
whappy_preds = wknn.predict(test_data)
print('r^2: ', metrics.r2_score(test_sln, whappy_preds))

r^2:  0.7555188584433628


In [26]:
#Decision Tree
from sklearn import tree
dt = tree.DecisionTreeRegressor(random_state = 41)
dt.fit(train_data,train_sln)
dt_happy_preds = dt.predict(test_data)
print('r^2: ', metrics.r2_score(test_sln, dt_happy_preds))

r^2:  0.6051757222282659


In [28]:
#Random Forest
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(random_state = 41)
rf.fit(train_data, train_sln)
rf_preds = rf.predict(test_data)
print('r^2: ', metrics.r2_score(test_sln, rf_preds))

r^2:  0.7926030280286398


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors = 5, weights = "uniform", metric = 'minkowski', p = 2)
classifier.fit(train_data, test_data)

ValueError: ignored

In [ ]:
# W-kNN

knn = KNeighborsClassifier(n_neighbors=5, weights = 'distance')

knn.fit(train_data, train_sln)

predictions = knn.predict(test_data)

print('Accuracy: ', metrics.accuracy_score(test_sln, predictions))

ValueError: ignored

In [ ]:
# DT
import sklearn
import pandas
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler

target= 'Country'
predictors = CleanRamen.columns.drop(target)
train_data, test_data, train_sln, test_sln = train_test_split(CleanRamen[predictors], CleanRamen[target], test_size = 0.2, random_state=41)
train_data.head()

,Review #,Brand,Variety,Style,Stars,Top Ten
1510,1070,HoMyeonDang,Premium Noodle House Pork & Seafood Flavor,Pack,4,0
2486,94,Mama,Artificial Chicken,Pack,2.5,0
1649,931,Kamfen,dried Mix Noodles Soya Bean Paste,Tray,3.75,0
611,1969,Nongshim,Noodle Soup Seasoned With Real Kimchi,Pack,4.75,0
261,2319,Nongshim,Saeutang Sabalmyeon,Bowl,3.5,0


In [ ]:
dt = tree.DecisionTreeRegressor()

dt.fit(train_data, train_sln)

predictions = dt.predict(test_data)

print('Accuracy: ', metrics.accuracy_score(test_sln, predictions))

ValueError: ignored

In [ ]:
# Normalization (Z-Score):

scaler = StandardScaler()
scaler.fit(train_data)

train_data_normalized = scaler.transform(train_data)
test_data_normalized = scaler.transform(test_data)

# run a knn
knn = KNeighborsClassifier(n_neighbors=13)

knn.fit(train_data_normalized, train_sln)

predictions = knn.predict(test_data_normalized)

print('Accuracy: ', metrics.accuracy_score(test_sln, predictions))

ValueError: ignored

# Results
After you conduct your learning experiment, summarize the results you got. Include visualizations (i.e. graphs) where appropriate.  How did you find the best k, n, m or other variables that you needed to tune. 

In [ ]:
# Include code to show your results (graphs, etc)
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets, neighbors
from mlxtend.plotting import plot_decision_regions

In [ ]:
# establish test_data and train_data here
shuffled_data = CleanRamen.sample(frac=1, random_state = 23)
test_data = CleanRamen.iloc[:800]
train_data = CleanRamen.iloc[800:]

In [ ]:
def knn_comparison(CleanRamen, k):
 x = CleanRamen[[test_data,train_data]].values
 y = CleanRamen["Country"].astype(int).values
 clf = neighbors.KNeighborsClassifier(n_neighbors=5)
 clf.fit(x, y)
# Plotting decision region
 plot_decision_regions(x, y, clf=clf, legend=2)
# Adding axes annotations
 plt.xlabel(test_data)
 plt.ylabel(train_data)
 plt.title('knn with K= + str(k'))
 plt.show()

SyntaxError: ignored

In [ ]:
def classifiers_all_kNN(test_data,train_data,k):
  
    results = []
    for i in range(len(test_data)):
      prediction = classifiers_kNN(test_data.iloc[i], train_data,k)
      results.append(prediction)
    return pandas.Series(results)

In [ ]:
import matplotlib.pyplot as plt
k_vals = [1, 3, 5, 7, 9, 21, 23, 25, 27, 29, 41, 53, 69, 101, 149]
kNN_acc = []

for k in k_vals:
  predictions = classifiers_all_kNN(test_data, train_data, k)
  current_acc = accuracy(test_data['Life expectancy'], predictions)
  kNN_acc.append(current_acc)

plt.suptitle('Country of Origin kNN Study', fontsize = 15)
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.plot(k_vals, kNN_acc, 'ro-', label = 'kNN')
plt.legend(loc = 'lower right', shadow =True)
plt.axis([0,150,0,100])

plt.show()

NameError: ignored

# Bumps in the Road:
What challenges did you encounter? How did you overcome these challenges? Did you have to adapt your strategy to account for these challenges? Why or why not?


I had many different challenges when working with this dataset, and I still have a lot to comb through with the model execution due to my interest in finding specific Countries that produce better ramen than others.  Some specific problems I had included separating and identifying the "Top Ten" ramen reviews from the NaN values and the "2016 #2" type examples. Another challenge I had was deciding what objective I wanted to find from this dataset.  Deciding if I wanted to do a classifier for the Country of origin, or regression for predicted rating (Stars) based on other predictors was something I went back and forth on. Lastly, converting the string values for brands into floats is a challenge I don't believe I have overcame.  I still have work to do on this ML Project, and I will be working in the next couple days to solve these issues.

# Conclusions
What insights/recommendations do you have? What did you find that was interesting? Which model was your best model, which models didn't work well? Why do you think this is? In general, I want a discussion of your experiment, the results, and what they mean.

When I originally did this ML program with a Regression to find the predicted Rating of a type of ramen based on the other predictors, I ran into many issues as one might tell as the accuracy of predicting the rating was not a telling predictor for the quality of ramen.  After switching to a Classifier, I had a better chance to find if there is quality differences based on Country of Origin.  From the model, the knn Classifier was the best model compared to Decision Trees and Random Forests.  I would appreciate some insight and feedback on how to solve these problems I'm running into.